In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd

In [2]:
from ipywidgets import interact, IntRangeSlider, ToggleButtons, Play

In [3]:
# A bit of customisation
matplotlib.rcParams['figure.figsize'] = (18.0, 5.0)

# %matplotlib inline
%config InlineBackend.figure_format='retina'

np.set_printoptions(precision=3, suppress=True)  # suppress scientific float notation

In [4]:
path = r'..'
# path = '.'
map_df = gpd.read_file("https://raw.githubusercontent.com/eurostat/Nuts2json/gh-pages/2016/3035/20M/2.json")

# Load data
Load all the files as a single dataset using xarray and its dask capabilities. In this way the files are not loaded in-memory but dask/xarray load them **only** when needed. `mask_and_scale` to True replaces array values equal to `_FillValue` with NA and scale values according to the formula `original_values * scale_factor + add_offset`.


In [5]:
ds = xr.open_mfdataset(path + '/*.nc', mask_and_scale=True)
ds

<xarray.Dataset>
Dimensions:  (region: 309, time: 341880)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 ... 2018-12-31T23:00:00
  * region   (region) object 'AL01' 'AL02' 'AL03' ... 'UKM8' 'UKM9' 'UKN0'
Data variables:
    t2m      (time, region) float32 dask.array<shape=(341880, 309), chunksize=(341880, 309)>

# Widget

The above selection method allows as to build custom interactive dashboards with a simple jupyter widgets through `ipywidget`. This process is very is simple. You just need to encapsulate you query in a function and decorate it with [interact](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html#Basic-interact) by specifying the desired range. Due to xarray and dask's out of core computation, querying and plotting this dataset is almost instant

In [6]:
@interact(var=list(ds.data_vars), 
          region=ds.coords['region'].values,
          month=(1,12),
          year=(1980,2018))
def plot_region_month(var, region, year, month):
    ds[var].sel(time='{}-{}'.format(year, month), region=region).plot(color='k') # year

interactive(children=(Dropdown(description='var', options=('t2m',), value='t2m'), Dropdown(description='region…

In [7]:
@interact(var=list(ds.data_vars), 
          region=ds.coords['region'].values,
          year_range=IntRangeSlider(min=1980,max=2018),
          timestep=ToggleButtons(options=[('week','w'),('month','m'),('year','a')]),
          continuous_update=False,
         )
def plot_region_month(var, region, year_range,timestep='m'):
    (ds[var].sel(region=region,
                 time=slice(str(year_range[0]), str(year_range[1])))
            .resample(time=timestep).mean()
            .plot(color='k')
    )

interactive(children=(Dropdown(description='var', options=('t2m',), value='t2m'), Dropdown(description='region…

Querying works also for a selection of regions

In [8]:
#Extract unique countries by taking the first two characters of all regions
countries = set(s[0:2] for s in ds.coords['region'].values)

@interact(var=list(ds.data_vars), 
          region=countries,
          month=(1,12),
          year=(1980,2018))
def plot_region_month(var, region, year, month):
    selection = [s for s in ds.coords['region'].values if s[0:2]==region]
    ds[var].sel(time='{}-{}'.format(year, month), region=selection).plot.line(x='time')

interactive(children=(Dropdown(description='var', options=('t2m',), value='t2m'), Dropdown(description='region…

Another example using geopandas. Here we plot the mean of each selected month/year per region

In [9]:
plot_min=dict(t2m=-10)
plot_max=dict(t2m=35)

@interact(var=list(ds.data_vars), 
          month=(1,12), 
          year=(1980,2018), # Play(value=1,min=1,max=12,step=1,description='Loop months'),
          continuous_update=False,)
def plot_region_month(var, year, month):
    fig, ax = plt.subplots(1, figsize=(14, 10))
    ax.axis('off')
    time_str = '{}-{}'.format(year, month)
    a = ds[var].sel(time=time_str).mean(dim='time').to_series().reset_index()
    annot_str = '{} \nMean Temp:${:.3g}^oC$'.format(time_str,a.mean().values[0])
    ax.text(.7,0.9,annot_str,fontdict={'fontsize':'xx-large'},transform=ax.transAxes)
    
    (map_df.merge(a, left_on='id', right_on='region')
                  .plot(column=var, cmap='coolwarm', vmin=plot_min.get(var,None),vmax=plot_max.get(var,None),
                        linewidth=0.3, edgecolor='0.4',
                        ax=ax,legend=True,)
                    )


interactive(children=(Dropdown(description='var', options=('t2m',), value='t2m'), IntSlider(value=1999, descri…